In [1]:
!pip install langchain_openai -q
!pip install langchain_community -q
!pip install langchain -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [40]:
!pip install -q langchain_openai langchain_community langchain


# **Create a Data Base**

In [49]:

import sqlite3




ModuleNotFoundError: No module named 'langchain_experimental'

In [4]:
cursor = connection.cursor()

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS teachers (
    teacher_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS subjects (
    subject_id INTEGER PRIMARY KEY AUTOINCREMENT,
    subject_name TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS teaches (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    teacher_id INTEGER,
    subject_id INTEGER,
    FOREIGN KEY (teacher_id) REFERENCES teachers(teacher_id),
    FOREIGN KEY (subject_id) REFERENCES subjects(subject_id)
)
""")

connection.commit()

In [6]:
teachers = [
    ("Supun", 42),
    ("Kamal", 29),
    ("Pubudu", 34)
]
cursor.executemany("INSERT INTO teachers (name, age) VALUES (?, ?)", teachers)

subjects = [
    ("Mathematics",),
    ("Chemestry",),
    ("Phycics",),
    ("English",)
]
cursor.executemany("INSERT INTO subjects (subject_name) VALUES (?)", subjects)

teaches = [
    (1, 1),
    (1, 2),
    (2, 3),
    (3, 4),
]
cursor.executemany("INSERT INTO teaches (teacher_id, subject_id) VALUES (?, ?)", teaches)

connection.commit()

In [7]:
cursor.execute("SELECT * FROM teachers")
teachers = cursor.fetchall()
print(teachers)

[(1, 'Supun', 42), (2, 'Kamal', 29), (3, 'Pubudu', 34)]


In [8]:
cursor.execute("""SELECT t.name
FROM teachers t
JOIN teaches te ON t.teacher_id = te.teacher_id
JOIN subjects s ON te.subject_id = s.subject_id
WHERE s.subject_name = 'Mathematics';""")
teachers = cursor.fetchall()
print(teachers)

[('Supun',)]


Initialize Long Chain SQL Database

In [9]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('API_KEY')

In [10]:
from langchain_community.utilities.sql_database import SQLDatabase

In [11]:
db = SQLDatabase.from_uri(f"sqlite:///school.db")

In [12]:
print(db.get_usable_table_names())
print(db.table_info)

['subjects', 'teachers', 'teaches']

CREATE TABLE subjects (
	subject_id INTEGER, 
	subject_name TEXT NOT NULL, 
	PRIMARY KEY (subject_id)
)

/*
3 rows from subjects table:
subject_id	subject_name
1	Mathematics
2	Chemestry
3	Phycics
*/


CREATE TABLE teachers (
	teacher_id INTEGER, 
	name TEXT NOT NULL, 
	age INTEGER NOT NULL, 
	PRIMARY KEY (teacher_id)
)

/*
3 rows from teachers table:
teacher_id	name	age
1	Supun	42
2	Kamal	29
3	Pubudu	34
*/


CREATE TABLE teaches (
	id INTEGER, 
	teacher_id INTEGER, 
	subject_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(teacher_id) REFERENCES teachers (teacher_id), 
	FOREIGN KEY(subject_id) REFERENCES subjects (subject_id)
)

/*
3 rows from teaches table:
id	teacher_id	subject_id
1	1	1
2	1	2
3	2	3
*/


# **Initialize LLM**

In [50]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4")


# **SQL Generation Chain**

In [46]:
from langchain.chains import create_sql_query_chain

query_generate = create_sql_query_chain(llm, db)

# **SQL Execution Tool**

In [47]:
from langchain_community.tools import QuerySQLDatabaseTool

query_execute = QuerySQLDatabaseTool(db=db)

In [56]:
query = query_generate.invoke({"question": "Details of all teachers"})
print(query)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [53]:
query_execute.invoke(query)

NameError: name 'query' is not defined

In [21]:
query = query_generate.invoke({"question": "Which teachers are assigned to teach Mathematics?"})
print(query)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [22]:
query_execute.invoke(query)

NameError: name 'query' is not defined

Answer Generator Chain

In [23]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given a user question, the generated SQL query, and its result, write a clear and natural answer to the question.

    User Question: {question}
    SQL Query: {query}
    SQL Result: {result}

    Final Answer:"""
)

final_answer = answer_prompt | llm | StrOutputParser()

In [24]:
chain = (
    RunnablePassthrough.assign(query=query_generate).assign(
        result=itemgetter("query") | query_execute
    )
    | final_answer
)

In [25]:
chain.invoke({"question": "Details of all teachers"})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [26]:
chain.invoke({"question": "Which teachers are assigned to teach Mathematics?"})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [55]:
!pip install -U langchain langchain-community langchain-openai

# Import necessary libraries
import os
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain.tools.sql.toolkit import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType

# ✅ Set your OpenAI API Key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# ✅ Setup the language model
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# ✅ Natural language to SQL prompt
prompt = PromptTemplate(
    input_variables=["question"],
    template="Convert this natural language question to SQL: {question}"
)

# ✅ Chain for generating SQL
query_generate = LLMChain(llm=llm, prompt=prompt)

# ✅ Generate SQL query
query = query_generate.invoke({"question": "Details of all teachers"})
print("Generated SQL:", query['text'])

# ✅ Connect to the database
db = SQLDatabase.from_uri("sqlite:///school.db")

# ✅ Create SQL execution agent
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
query_execute = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# ✅ Execute the query
response = query_execute.invoke(query['text'])
print("Response:", response)


ModuleNotFoundError: No module named 'langchain.tools.sql'